In [1]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [2]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "PESEL",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [3]:
employees_df

,id,imię,nazwisko,PESEL,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Marcin,Habrukovich,62010614411,Bielsko-Biała,ul. Kozielska 20/2,48658691568,marcin.habrukovich@kuba.pl,2014-01-01,None,menadżer,64.4
1,2,Katarzyna,Inozemtseva,95022654380,Wrocław,ul. Rodacka 31,48369975059,katarzyna.inozemtseva@kuba.pl,2014-01-01,None,sprzedawca,56.9
2,3,Zofia,Kopeć,78120203124,Wrocław,ul. Ofiar Bełżca 50/34,48714250382,zofia.kopec@kuba.pl,2014-01-01,None,sprzedawca,49.2
3,4,Alfreda,Kowal,67062839063,Wrocław,ul. Kasztanowa 7,48350177921,alfreda.kowal@kuba.pl,2014-01-01,None,mechanik,53.7
11,5,Zenon,Tiba,78062927931,Wrocław,ul. Leśna 85,48962332251,zenon.tiba@kuba.pl,2014-01-01,None,informatyk,59.8
4,6,Hanna,Nakonieczna,90060401600,Sulechów,ul. Oliwna 95/19,48139123974,hanna.nakonieczna@kuba.pl,2014-11-29,None,mechanik,45.2
5,7,Wojciech,Zguda,61030891435,Wrocław,ul. Mikołaja Kopernika 55/27,48065713107,wojciech.zguda@kuba.pl,2014-12-03,2015-04-29,mechanik,45.2
6,8,Konrad,Grzyb,80031703914,Sopot,ul. Plac Niepodległości 22/20,48902801010,konrad.grzyb@kuba.pl,2015-06-07,None,pomocnik mechanika,36.0
7,9,Marta,Frątczak,85052442743,Wrocław,ul. Ogrodowa 88,48192024930,marta.fratczak@kuba.pl,2015-06-26,None,diagnosta samochodowy,58.5
8,10,Ivan,Siwiński,64121031518,Poznań,ul. Wyżyny 64/16,48669257477,ivan.siwinski@kuba.pl,2015-10-27,None,diagnosta samochodowy,58.9


In [4]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [5]:
no_transactions = np.random.poisson(2000)
begining_buy = 20

dates = random_transaction_dates(begining_date, end_date, no_transactions)

In [6]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [7]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)
year_ceil = [int(date[:4])-19 for date in dates[:20]]

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()
not_fixed_our_cars = our_cars[:]

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
            jp += 1
            year_ceil.append(int(dates[i][:4])-19)
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if len(not_fixed_our_cars) >= 1 and np.random.rand() <= weights[1]: #fixing car we currently have
           client_ids[i] = 0
           car_ids[i] = np.random.choice(not_fixed_our_cars)
           not_fixed_our_cars.remove(car_ids[i])
        elif np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

In [8]:
len(year_ceil), len(pd.unique(client_ids))

(1388, 1389)

In [9]:
year_ceil

[1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,

In [10]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [11]:
len(buy_indexes[0]), len(sell_indexes[0]), len(service_indexes[0])

(755, 749, 466)

In [12]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [13]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [14]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [15]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [16]:
year_floor = 1960
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])

menager = employees_df.iloc[0]
if int(menager["PESEL"][-2]) % 2 == 1:
    menager_sex = "M"
else:
    menager_sex = "K"
clients_df.loc[0] = [0, menager["imię"], menager["nazwisko"], menager_sex, menager["PESEL"], menager["miasto"], menager["adres"], False, menager["telefon"], menager["email"]]

unique_clients = np.unique(client_ids)
unique_clients = unique_clients[unique_clients != 0]
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Marcin,Habrukovich,M,62010614411,Bielsko-Biała,ul. Kozielska 20/2,False,48658691568,marcin.habrukovich@kuba.pl
1,1,Wiktoria,Abdel Massih,K,94111357357,Skoki,ul. Jana Pawła II 15,False,48561986054,wikabdel mas6435@gmail.com
2,2,Rafał,Drzewicki,M,73061642456,Wrocław,ul. Lipowa 84/43,False,48443164015,dr.rafal@o2.pl
3,3,Ryszard,Tomkowiak,M,94100741295,Lubin,ul. Aleksandra Gierymskiego 24,False,48395404939,k7samt9md@opayq.com
4,4,Ingrida,Chruszczyk,K,74011162091,Warszawa,ul. Kwiatowa 47,False,48538679656,ichruszczyk@onet.pl
...,...,...,...,...,...,...,...,...,...,...
1384,1384,Małgorzata,Kobusińska,K,94101460052,Ostrowiec Świętokrzyski,al. Aleja Kasztanowa 33/8,False,48471669658,m.kobusinska@gmail.com
1385,1385,Ewa,Ciesielska,K,75080414117,Wrocław,ul. Tadeusza Kościuszki 20,False,48388969123,ciesielska19751@gmail.com
1386,1386,Lela,Orzeł,K,63052844178,Radomsko,ul. Wesoła 30/22,False,48985929043,u12lct564s6ujv@opayq.com
1387,1387,Anna,Ilnicka,K,None,Łódź,ul. Gryfińska 19/9,False,48479937554,a-ilnicka7263@gmail.com


In [17]:
clients_with_lc = clients_df[clients_df["karta_lojalnościowa"]]["id"].values

loyalty_card_dates = [() for i in range(len(clients_with_lc))]
for (i, client) in enumerate(clients_with_lc):
    loyalty_card_dates[i] = client, dates[np.where(client_ids == client)[0][2]]


In [18]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Marcin,Habrukovich,M,62010614411,Bielsko-Biała,ul. Kozielska 20/2,False,48658691568,marcin.habrukovich@kuba.pl
1,1,Wiktoria,Abdel Massih,K,94111357357,Skoki,ul. Jana Pawła II 15,False,48561986054,wikabdel mas6435@gmail.com
2,2,Rafał,Drzewicki,M,73061642456,Wrocław,ul. Lipowa 84/43,False,48443164015,dr.rafal@o2.pl
3,3,Ryszard,Tomkowiak,M,94100741295,Lubin,ul. Aleksandra Gierymskiego 24,False,48395404939,k7samt9md@opayq.com
4,4,Ingrida,Chruszczyk,K,74011162091,Warszawa,ul. Kwiatowa 47,False,48538679656,ichruszczyk@onet.pl
...,...,...,...,...,...,...,...,...,...,...
1384,1384,Małgorzata,Kobusińska,K,94101460052,Ostrowiec Świętokrzyski,al. Aleja Kasztanowa 33/8,False,48471669658,m.kobusinska@gmail.com
1385,1385,Ewa,Ciesielska,K,75080414117,Wrocław,ul. Tadeusza Kościuszki 20,False,48388969123,ciesielska19751@gmail.com
1386,1386,Lela,Orzeł,K,63052844178,Radomsko,ul. Wesoła 30/22,False,48985929043,u12lct564s6ujv@opayq.com
1387,1387,Anna,Ilnicka,K,None,Łódź,ul. Gryfińska 19/9,False,48479937554,a-ilnicka7263@gmail.com


In [19]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [20]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(64)

In [21]:
np.sum(pd.isna(clients_df["PESEL"]))

np.int64(209)

# auta

In [22]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [23]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [24]:
df_samochody = create_car_table(max(car_ids))

In [25]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Peugeot,Rifter,4,2004,Hatchback,False,czarny,Diesel,2.0,131,Automatyczna,5,5,False
1,2,Mitsubishi,Eclipse Cross,4,2012,SUV,False,srebrny,Benzyna,2.3,163,Automatyczna,5,5,False
2,3,Subaru,XV,4,2008,Kombi,False,srebrny,Benzyna,2.5,147,Manualna,3,5,False
3,4,Toyota,Aygo,4,2007,SUV,False,biały,Benzyna,1.1,72,Manualna,3,2,False
4,5,Kia,Sportage,4,2007,SUV,True,czarny,Diesel,2.8,184,Manualna,3,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,986,Hero,Hunk,2,2007,,False,czarny,Benzyna,0.6,89,Manualna,0,1,True
986,987,Fiat,500X,4,2013,Van,True,czarny,Benzyna,1.8,110,Manualna,5,5,True
987,988,Opel,Corsa,4,1999,Sedan,False,czarny,Benzyna,1.4,87,Manualna,3,5,False
988,989,Skoda,Octavia,4,2003,SUV,False,biały,Benzyna,1.7,105,Automatyczna,5,5,False


In [26]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [27]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Automatyczna', 'Manualna', 'Półautomatyczna'], dtype=object)

# Usługi

In [28]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [29]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [30]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [31]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_21876\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2009' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [32]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [33]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,3,2014-01-02,36800.0
1,2,2,2,3,2014-01-03,108800.0
2,3,3,3,3,2014-01-04,47200.0
3,4,4,4,2,2014-01-04,27800.0
4,5,5,5,2,2014-01-04,46400.0
...,...,...,...,...,...,...
750,751,985,1162,2,2017-12-22,53100.0
751,752,986,1385,2,2017-12-24,26100.0
752,753,987,1386,3,2017-12-26,39300.0
753,754,988,18,3,2017-12-26,19500.0


In [34]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [35]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [36]:
loyalty_card_customer_ids = [el[0] for el in loyalty_card_dates]
loyalty_card_dates_sep = [el[1] for el in loyalty_card_dates]

In [37]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:
        date_index = loyalty_card_customer_ids.index(customer_id)
        if loyalty_card_dates_sep[date_index] < df_sell.loc()[i]['data_zaksięgowania']:
            sell_prices[i] *= 0.97
            #print(i)

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
#print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_21876\1822833189.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:


In [38]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,19,21,3,62832.0,6,2014-01-23
1,2,20,22,2,63138.0,6,2014-01-24
2,3,4,23,3,36700.0,1,2014-01-25
3,4,11,24,2,84032.0,12,2014-01-25
4,5,1,25,2,50336.0,12,2014-01-25
...,...,...,...,...,...,...,...
744,745,978,1227,2,18512.0,12,2017-12-26
745,746,981,50,2,55406.4,6,2017-12-26
746,747,988,1203,2,23600.0,1,2017-12-26
747,748,984,21,2,72300.0,1,2017-12-29


In [39]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

np.int64(0)

In [40]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [41]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(np.int64(21), array([ 1, 30])), (np.int64(26), array([19])), (np.int64(30), array([63, 61])), (np.int64(34), array([44, 62])), (np.int64(35), array([54])), (np.int64(36), array([ 4, 48])), (np.int64(37), array([ 8, 27])), (np.int64(43), array([42, 40])), (np.int64(44), array([10])), (np.int64(45), array([30, 55])), (np.int64(46), array([21])), (np.int64(47), array([32])), (np.int64(49), array([45])), (np.int64(50), array([56])), (np.int64(51), array([62,  5])), (np.int64(54), array([54])), (np.int64(57), array([ 2, 43])), (np.int64(58), array([41])), (np.int64(60), array([25])), (np.int64(59), array([27, 60, 15])), (np.int64(63), array([12])), (np.int64(64), array([6])), (np.int64(66), array([39])), (np.int64(68), array([35])), (np.int64(67), array([49])), (np.int64(71), array([6])), (np.int64(72), array([10, 54, 27])), (np.int64(73), array([49])), (np.int64(76), array([25, 39])), (np.int64(75), array([19])), (np.int64(77), array([23])), (np.int64(78), array([35])), (np.int64(86), ar

In [42]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [43]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Zderzak,SEAT Leon czarny,243,1,1
1,Dywan,SEAT Leon,15,1,2
2,Dystrybutor,Hyundai Ioniq,295,2,1
3,Lusterko wewnętrzne,Nissan Pulsar,23,3,1
4,Lusterko zewnętrzne lewe,Nissan Pulsar szary,27,3,1
...,...,...,...,...,...
709,Rozrusznik,Opel Corsa,280,462,1
710,Lakier,biały,233,463,1
711,Oświetlenie boczne,Honda Activa 5G,38,464,1
712,Podświetlenie tablicy,,20,465,1


# Wyposażenie

In [44]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [45]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Zderzak,SEAT Leon czarny,243.0
1,2,Dywan,SEAT Leon,15.0
2,3,Dystrybutor,Hyundai Ioniq,295.0
3,4,Lusterko wewnętrzne,Nissan Pulsar,23.0
4,5,Lusterko zewnętrzne lewe,Nissan Pulsar szary,27.0
...,...,...,...,...
705,507,Drzwi lewe przednie,Chevrolet Cruze czerwony,542.0
709,508,Rozrusznik,Opel Corsa,280.0
710,509,Lakier,biały,233.0
711,510,Oświetlenie boczne,Honda Activa 5G,38.0


# Usługi - uzupełnienie

In [46]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [47]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,21,32,7,2014-02-04,360
1,2,26,0,6,2014-02-12,390
2,3,30,56,6,2014-02-14,70
3,4,34,0,6,2014-02-16,670
4,5,35,66,7,2014-02-18,800
...,...,...,...,...,...,...
461,462,976,1378,6,2017-12-13,370
462,463,977,766,7,2017-12-14,310
463,464,974,0,6,2017-12-16,50
464,465,980,1380,7,2017-12-17,30


In [48]:
df_services.to_csv("outputs/usługi.csv", index=False)

# Użyte części - uzupełnienie

In [49]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [50]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,1
1,2,1,2,2
2,3,2,3,1
3,4,3,4,1
4,5,3,5,1
...,...,...,...,...
709,710,462,508,1
710,711,463,509,1
711,712,464,510,1
712,713,465,43,1


In [51]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [52]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    #if price > 1000:
    #    quantity += rng.random() < 0.5
    #elif price > 500:
    #    quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    #else:
    #    quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(price * rng.uniform(0.75, 0.95))[0]
    service_date = get_service_date(part_id)
    
    row = [int(i+1), int(part_id), total_cost, int(quantity)]
    parts_purchase_pd.loc[i] = row


In [53]:
parts_purchase_pd

,id,id_części,koszt,ilość
0,1.0,1.0,228.0,1.0
1,2.0,2.0,12.0,2.0
2,3.0,3.0,226.0,1.0
3,4.0,4.0,18.0,1.0
4,5.0,5.0,20.0,1.0
...,...,...,...,...
506,507.0,507.0,468.0,1.0
507,508.0,508.0,238.0,1.0
508,509.0,509.0,200.0,1.0
509,510.0,510.0,29.0,1.0


In [54]:
parts_purchase_pd.to_csv("outputs/zakup_części.csv", index=False)

In [55]:
df_equipment = df_equipment[['id', 'nazwa', 'atrybut']]

In [56]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)